In [4]:
import multiprocessing
import os
import numpy as np
from numpy import linalg as la
import sklearn
from sklearn.neighbors import radius_neighbors_graph, kneighbors_graph
from sklearn.preprocessing import MinMaxScaler
from sklearn import cluster,datasets, mixture
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from itertools import cycle, islice
import pandas
import scipy
from scipy.stats import norm
from PIL import Image

m=1500

In [5]:
def getY(labels):
    Y = np.zeros((len(labels), max(labels)+1))
    for i in range(0, len(labels)):
        Y[i, labels[i]] = 1
    return Y

ZZ Top Bound

In [21]:
def logZZTop(y1,y2,U,out=False):
    y = (y1+y2)>0
    U = U[y,:]
    norms = la.norm(U,axis = 0)
    norms[norms==0]=1
    U = U/norms
    U = U-np.mean(U,axis=0)
    y1 = y1[y]
    y2 = y2[y]
    σ2 = np.mean(U*U)
    if out: print('sigma='+str(σ2))
    t = max(la.norm(y1@U)**2/np.sum(y1), la.norm(y2@U)**2/np.sum(y2))-σ2*U.shape[1]
    if out: print('t='+str(t))
    return np.min(np.log(U.shape[0]) -t**2/(σ2*U.shape[1] +t/3)/2,0)

# SpecialK

In [33]:
def specialK(W, n, α, kmax=5, out=False):
    Lambda, V = scipy.sparse.linalg.eigsh(W, k=n, which="LM")
    Lambda, D = np.absolute(Lambda), np.absolute(V)
    D = D*Lambda**(0.5)
    r=2
    Yopt = np.ones(D.shape[0])
    while r<=kmax:
        if out: print('============== r= '+str(r)+'============')
        kmeans = KMeans(n_clusters=r)
        kmeans.fit(D[:,np.argsort(la.norm(D,axis=0))[::-1][0:50]])
        Y = getY(kmeans.labels_)
        topk=int(min(r*(r-1)/2,10)) # top 10 pairs of clusters, having the highest cut
        cuts = Y.T@W@Y
        idx = np.triu_indices(r,1)
        for s in np.argpartition(cuts[idx],-topk, axis=None)[-topk:]:
            if out: print("---- s1="+str(idx[0][s])+', s2='+str(idx[1][s])+", cut="+str(cuts[idx][s]))
            logZZ = logZZTop(Y[:,idx[0][s]],Y[:,idx[1][s]],D, out=out)
            if out: print('ZZ Top: '+ str(np.exp(logZZ)))
            if logZZ> np.log(α):
                    if out: print('k = '+ str(r-1))
                    return Yopt
                
        Yopt = Y
        r=r+1
    return Yopt

# Make some Experiments

Functions to create the similarity matrices.

In [24]:
def get_W_dense(data, desired_percentage=0.99,knn= 10,eps_min= 0.1,eps_max= 1.0,eps_step= 0.01):
    dist_matrix = sklearn.metrics.pairwise.pairwise_distances(data)
    eps_candidate = eps_min
    eps = 0
    while eps==0:
        nr_core_points = np.sum(np.sum(dist_matrix < eps_candidate,1) >= knn)
        #print(str(eps_candidate)+' perc: '+str(nr_core_points / len(dist_matrix)))
        if (nr_core_points / len(dist_matrix)) >= desired_percentage:
            eps=eps_candidate
        else:
            eps_candidate = eps_candidate+eps_step
        if eps_candidate>eps_max:
            raise ValueError("Desired percentage is not achievable.")
    W = radius_neighbors_graph(data, radius=eps)
    return W
def get_W_cut(data):
    W = kneighbors_graph(data, n_neighbors=10, mode="distance", include_self=False, n_jobs=-1)
    W = 0.5*(W + W.T)
    d = np.sum(W, axis=1)
    d[d== 0] =1
    d = np.power(d,-0.5)
    D= scipy.sparse.diags(np.squeeze(np.asarray(d)))
    return D@W@D

Functions to generate data: 

In [25]:
def generateCircles(n):
    scaler = MinMaxScaler(feature_range=(0, 3))
    circles, labels = datasets.make_circles(n_samples=m, factor=.5, noise=n)
    scaler.fit(circles)
    return "circles", scaler.transform(circles), labels, 2
def generateMoons(n):
    scaler = MinMaxScaler(feature_range=(0, 3))
    moons, labels = datasets.make_moons(n_samples=m, noise=n)
    scaler.fit(moons)
    return "moons", scaler.transform(moons), labels, 2
def generateBlobs(n):
    scaler = MinMaxScaler(feature_range=(0, 3))
    blobs, labels = datasets.make_blobs(n_samples=m, centers= [[1,1],[4,9],[6,4]], cluster_std=[n + 0.5, n + 1.25, n + 0.25])
    scaler.fit(blobs)
    return "blobs", scaler.transform(blobs), labels, 3
def generateRandom(n):
    scaler = MinMaxScaler(feature_range=(0, 3))
    no_structure, labels = np.random.rand(m, 2), np.zeros(m)
    scaler.fit(no_structure)
    return "random", scaler.transform(no_structure), labels, 1

In [30]:
noise = 0.05
shape, data, labels, k = generateCircles(n=noise)
W_cut = get_W_cut(data)
W_dense = get_W_dense(data)

SpecialK of $W_C$.

In [34]:
specialK(W_cut,200,1e-2, out=True)

============== r= 2============
---- s1=0, s2=1, cut=0.0
sigma=0.000541787791100833
t=18.169140646754368
ZZ Top: 3.531820870427315e-09
============== r= 3============
---- s1=0, s2=1, cut=0.0
sigma=0.0006784220752203737
t=23.008965721979674
ZZ Top: 2.214521639722002e-12
---- s1=0, s2=2, cut=4.734471408828129
sigma=0.0008370717663834261
t=7.570498741003118
ZZ Top: 0.017788899261507556
k = 2


array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]])

SpecialK of $W_R$.

In [35]:
specialK(W_dense,200,1e-2, out=True)

============== r= 2============
---- s1=0, s2=1, cut=60.0
sigma=0.0005130052252738182
t=22.890708034697155
ZZ Top: 2.897238814401089e-12
============== r= 3============
---- s1=0, s2=1, cut=0.0
sigma=0.0007293455444426975
t=36.40483497821426
ZZ Top: 3.62289942433069e-21
---- s1=0, s2=2, cut=80.0
sigma=0.0011092788635083206
t=6.29870334325437
ZZ Top: 0.1470365750264144
k = 2


array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]])